In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/full-program-list/PROD_Full_Program_List_Published.csv
/kaggle/input/country-and-university-data/country_university.csv
/kaggle/input/country/country.csv
/kaggle/input/dataset-thesis7/RAW_StudeeDB_AppFormData_Thesis7 (1).csv
/kaggle/input/program-list-complete/Program_List_Complete.csv
/kaggle/input/thesis-datasets/RAW_StudeeDB_Nationalities_Thesis2.csv
/kaggle/input/thesis-datasets/RAW_StudeeDB_AdmissionsCriteria_Thesis6.csv
/kaggle/input/thesis-datasets/RAW_StudeeDB_StudentClicks_Thesis3.csv
/kaggle/input/thesis-datasets/RAW_StudeeDB_UniversityData_Thesis5.csv
/kaggle/input/thesis-datasets/RAW_StudeeDB_ApplicationData_Thesis1.csv
/kaggle/input/applicationdata-application-student-and-program-id/Result_4.csv
/kaggle/input/full-program-list-including-removed-programs/Program_List_Including_Archived.csv


In [3]:
!pip install scikit-surprise

In [4]:
import numpy as np
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder  # Added OneHotEncoder import
import os
from sklearn.metrics.pairwise import cosine_similarity

# Load the datasets
df_application = pd.read_csv('/kaggle/input/thesis-datasets/RAW_StudeeDB_ApplicationData_Thesis1.csv')
df_clicks = pd.read_csv('/kaggle/input/thesis-datasets/RAW_StudeeDB_StudentClicks_Thesis3.csv')
df_university = pd.read_csv('/kaggle/input/thesis-datasets/RAW_StudeeDB_UniversityData_Thesis5.csv')
df_full_program_list = pd.read_csv('/kaggle/input/program-list-complete/Program_List_Complete.csv')
df_app_student_program_uuid = pd.read_csv('/kaggle/input/applicationdata-application-student-and-program-id/Result_4.csv')
df_country = pd.read_csv('/kaggle/input/country-and-university-data/country_university.csv')

In [5]:
print("Country")
print(df_country.head())

Country
                              city_uuid                          country_uuid  \
0  3eacc079-fd1e-4d94-8b8e-d1d318d96662  01c7c46d-4fb4-4010-a0b8-35a5619c68ba   
1  92719895-3dc1-487d-bfc0-5bbab093da65  01c7c46d-4fb4-4010-a0b8-35a5619c68ba   
2  a413882c-86ac-40e5-be6b-4ab9e6141d97  01c7c46d-4fb4-4010-a0b8-35a5619c68ba   
3  b8fc2943-0947-41c8-0723-da9e0975293d  01c7c46d-4fb4-4010-a0b8-35a5619c68ba   
4  d6908893-51b8-4d55-bc26-f177b9be7059  01c7c46d-4fb4-4010-a0b8-35a5619c68ba   

  country_name                       university_uuid  
0       Sweden  378da73d-31c6-4248-fe9b-e8c186345981  
1       Sweden  e4e81829-d379-4310-9db8-e6a4188aced0  
2       Sweden  668c3472-5488-47b0-a529-2e14fd1eb0c2  
3       Sweden  40677364-b6ea-4c29-da5d-b9fea3478d0b  
4       Sweden  fea045a4-1d85-40cd-a2e6-9aa75a2eca23  


In [6]:
print("Application Data")
print(df_application.head())

Application Data
  application_account_url       BIN_TO_UUID(`application_uuid`)  \
0                     NaN  233554bf-f8bd-4992-942a-981d8d10c9a3   
1                     NaN  3c0d1548-7d33-46c1-9014-127489d54eea   
2                     NaN  007c863c-3b79-44b3-8577-8aef521a623c   
3                     NaN  7d42783a-2233-45c1-b71c-7f20532717ad   
4                     NaN  21aa66c2-2ba4-45e6-9ce4-1220737e8f7c   

     created_date_time   BIN_TO_UUID(`education_agent_uuid`)  \
0  2022-07-30 07:48:34                                   NaN   
1  2023-08-25 11:29:15  34003144-3725-4846-8578-bd7af0373910   
2  2023-10-09 08:40:55                                   NaN   
3  2024-07-21 06:14:49                                   NaN   
4  2022-09-02 10:16:50                                   NaN   

                         email  first_name  \
0     Merdimakiadi10@gmail.com       Merdi   
1        proffababio@yahoo.com      EGBUKA   
2  gideonrockson1999@gmail.com      Gideon   
3   mothibi

In [7]:
print("Student Clicks Data")
print(df_clicks.head())

Student Clicks Data
  BIN_TO_UUID(`city_uuid`)             BIN_TO_UUID(`click_uuid`)  \
0                      NaN  000050ed-8f99-4035-b0d7-574560ee6dd8   
1                      NaN  00037b21-71b5-43b6-bb3e-68208919ed0d   
2                      NaN  0003f992-3ed1-4042-8ad0-ff2c6b436451   
3                      NaN  000493bd-ea6a-4821-b9b4-797102eaa3c6   
4                      NaN  00052ed0-f59e-47b5-b484-1844c4fadb67   

  BIN_TO_UUID(`country_uuid`)    created_date_time isas_processed_date_time  \
0                         NaN  2022-07-22 07:11:09                      NaN   
1                         NaN  2022-02-06 21:55:19      2022-02-11 06:39:20   
2                         NaN  2021-01-06 01:18:29      2021-05-12 11:29:21   
3                         NaN  2021-03-08 05:48:03      2021-05-12 11:29:21   
4                         NaN  2022-09-19 08:54:28                      NaN   

      BIN_TO_UUID(`program_level_uuid`)           BIN_TO_UUID(`program_uuid`)  \
0              

In [8]:
# Rename columns to ensure consistency
df_clicks = df_clicks.rename(columns={'BIN_TO_UUID(`student_uuid`)': 'Student_UUID'})

In [9]:
print("University Data")
print(df_university.head())

University Data
               BIN_TO_UUID(`city_uuid`)          BIN_TO_UUID(`currency_uuid`)  \
0  b6ee89dc-cea9-4bed-9436-93843b79b050  37ac0c29-7f04-4adc-be67-cdf087371ee5   
1  528da28a-51dc-4444-eeb0-debae73e3fba  023e8e35-7d2a-40f0-befd-9022186ad8cc   
2  69865792-405d-4930-20a5-e8ff078d3d91  0dc1b638-c4d0-4dfb-b675-246e35e677d1   
3  46ecdca2-a396-4d75-81d5-5b135d31f92f  023e8e35-7d2a-40f0-befd-9022186ad8cc   
4  04fc689f-6cf5-4284-84c7-06a1c09eec1f  0dc1b638-c4d0-4dfb-b675-246e35e677d1   

  BIN_TO_UUID(`default_university_contact_uuid`)  is_commercial  \
0           670c3cb4-e363-4700-963e-9e82d228ed24           True   
1           574b5304-7240-4a78-a443-c54b96179090           True   
2           5ee34e02-4f65-4a7a-8cb4-8d023986cdb1           True   
3           20ecb5d5-f9fa-4cec-951b-1b989df23fcf           True   
4                                            NaN           True   

  is_deposit_required  university_acceptance_rate  university_allows_agents  \
0             U

In [10]:
# Correct the typo in the df_university dataset
df_university = df_university.rename(columns={'universitu_uuid': 'university_uuid'})

In [11]:
print(df_full_program_list.head())

                              program_name  \
0                        Cognitive Science   
1          Communication and Media Studies   
2                   Computing and Business   
3  Global Financial Management and Systems   
4                         Professional MBA   

                           program_uuid  \
0  cf1fdd63-2c40-4779-9f7f-214720e1b2c7   
1  2bbb3339-8c36-4bff-ad76-c410fbcb110e   
2  a55cf874-3778-42c3-99c0-b4b40d94ba2e   
3  59c55a3b-bb64-4d27-b12f-327bc6ba81c3   
4  761e3bec-bd24-4445-aeca-5d4960fc7ab3   

              program_studee_friendly_name program_tuition_fees_per_year  \
0                        Cognitive Science                         31796   
1          Communication and Media Studies                         31796   
2                   Computing and Business                         35705   
3  Global Financial Management and Systems                         37694   
4                         Professional MBA                         10206   

  progr

In [12]:
print(df_app_student_program_uuid.head())

                       Student_App_UUID                          Student_UUID  \
0  0000eb21-655d-4bd6-8641-e7f5898e35a3  ffd9efba-0b60-496e-b16f-f39fbb7c7832   
1  00026801-704a-4518-87b6-1ed5a501f4d8  f51da60a-0b64-4c5e-991c-9f951ce54566   
2  0007d663-f56d-423d-abe3-ebe7e3c4f702  0f63919b-fa66-46f3-a67d-68ba22c181d3   
3  0008ab69-3760-4767-8ef6-b7dedda479b4  3b0dcaa6-99cc-4c2e-a153-b5b2f786fb32   
4  0008c4a3-dbd0-4ae7-bf67-59716e4ab077  cc4e91e7-4e00-444e-9d95-f3a186e1aff6   

                           Program_UUID                   Program_Intake_UUID  
0  6f082f72-51be-4944-baf4-41b4f1a81202  5db6e13f-8d81-11ec-b755-02cd10dd1184  
1  f7fb376d-9749-46be-80f8-14bdfca24e6b  603e2492-8d81-11ec-b755-02cd10dd1184  
2  7a91acab-67d7-4c5a-bf68-53a0e8200ebc  aba5f8f6-abc6-4a45-9793-1f85a4956616  
3  331545b5-45b9-4591-9db0-5e0ec109a4be  a9e89c9e-c6c9-4f04-acd8-fafbd0e1a42d  
4  c136aedf-d866-4beb-9bbf-a85f0b00e499  5f40db61-8d81-11ec-b755-02cd10dd1184  


In [13]:
# Merge the programs dataset with the application program UUID dataset on 'Program_UUID'
df_merged_programs_app = pd.merge(df_app_student_program_uuid, df_full_program_list, 
                                  how='inner', left_on='Program_UUID', right_on='program_uuid')

#Merge the programs with the clicks dataset
df_merged_programs_app = pd.merge(df_merged_programs_app, df_clicks, how='inner', on='Student_UUID')


# Ensure 'BIN_TO_UUID(`university_uuid`)' is renamed to 'university_uuid' in both dataframes
df_university = df_university.rename(columns={'BIN_TO_UUID(`university_uuid`)': 'university_uuid'})
df_merged_programs_app = df_merged_programs_app.rename(columns={'BIN_TO_UUID(`university_uuid`)': 'university_uuid'})

# Drop redundant 'university_uuid' columns from df_merged_programs_app if they exist
df_merged_programs_app = df_merged_programs_app.loc[:, ~df_merged_programs_app.columns.duplicated()]

df_merged_programs_app = pd.merge(df_merged_programs_app, df_university, how='left', on='university_uuid')

df_merged_programs_app.head()

,Student_App_UUID,Student_UUID,Program_UUID,Program_Intake_UUID,program_name,program_uuid,program_studee_friendly_name,program_tuition_fees_per_year,program_delivery_method,state,...,university_allows_agents,university_decision_period,university_location_address,university_number_of_international_students,university_number_of_nationalities,university_number_of_students,university_rank,university_type,_BATCH_ID__y,_BATCH_LAST_RUN__y
0,0000eb21-655d-4bd6-8641-e7f5898e35a3,ffd9efba-0b60-496e-b16f-f39fbb7c7832,6f082f72-51be-4944-baf4-41b4f1a81202,5db6e13f-8d81-11ec-b755-02cd10dd1184,Business Administration Diploma,6f082f72-51be-4944-baf4-41b4f1a81202,Business Administration,18130,On campus,published,...,True,4,2300 Ryan Road Courtenay BC V9N 8N6,367.0,40.0,9000.0,8936.0,public,2.0,2024-09-02 11:09:04
1,0027bdd9-8878-414d-b106-f00c03d6196b,c371d879-4eb3-431e-95bb-6489f5c36fc5,e20fe2b1-5517-4761-a401-6acad6aaa3c7,5fdc6e05-8d81-11ec-b755-02cd10dd1184,Project Management,e20fe2b1-5517-4761-a401-6acad6aaa3c7,NaN,9386,On campus,archived,...,False,4,"3737 Wascana Pkwy, Regina, SK S4S 0A2, Canada",3000.0,90.0,16600.0,1478.0,public,2.0,2024-09-02 11:09:04
2,00462585-7b4d-49ad-81f1-3ddcd1a04ac5,ed0a6684-b8c1-4eb9-86f1-36493c9b4e1b,c0d94ff7-7378-4f49-b72e-e4fffc1ae189,ec3b723e-47a4-412e-b3eb-83909273a58b,Certificate in Youth Work Practice,c0d94ff7-7378-4f49-b72e-e4fffc1ae189,Youth Work Practice,3000,Blended learning,published,...,True,4,"950 Main St, Worcester, MA 01610, USA",433.0,80.0,3095.0,805.0,private,2.0,2024-09-02 11:09:04
3,00b6a109-8634-414d-a405-3d3d920bf1ff,93dfada5-27a5-450d-9b0a-be2866634731,0b61620d-b2ff-473b-87bb-00b0a33f3997,5bd971e4-8d81-11ec-b755-02cd10dd1184,Business Administration: Business Analytics,0b61620d-b2ff-473b-87bb-00b0a33f3997,Business Administration: Business Analytics,8640,On campus,published,...,True,4,"102 Curris Center, Murray, KY 42071, United St...",401.0,45.0,10000.0,2287.0,public,2.0,2024-09-02 11:09:04
4,00c1e5c2-8fa3-4da1-a803-a3e9adfb2d7e,17415f2c-c9f9-4517-a9fd-64a7d6f455fd,ebc763f9-2f33-45b4-9d49-e342d80ccf08,ee5490e6-c58f-4927-b027-fb2c9d913319,Public Relations – Corporate Communications,ebc763f9-2f33-45b4-9d49-e342d80ccf08,Public Relations: Corporate Communications,17789,On campus,published,...,True,4,"941 Progress Ave, Scarborough, ON M1G 3T8, Ca...",11000.0,126.0,22000.0,4696.0,public,2.0,2024-09-02 11:09:04


In [14]:
# Rename the university_uuid column in both dataframes if necessary
df_merged_programs_app = df_merged_programs_app.rename(columns={'BIN_TO_UUID(`university_uuid`)': 'university_uuid'})
df_country = df_country.rename(columns={'BIN_TO_UUID(`university_uuid`)': 'university_uuid'})

# Drop redundant 'university_uuid' columns from df_merged_programs_app if they exist
df_merged_programs_app = df_merged_programs_app.loc[:, ~df_merged_programs_app.columns.duplicated()]

# Now perform the merge with the country dataset
df_merged_programs_app = pd.merge(df_merged_programs_app, df_country, how='inner', on='university_uuid')

# Check the merged result
df_merged_programs_app.head()




,Student_App_UUID,Student_UUID,Program_UUID,Program_Intake_UUID,program_name,program_uuid,program_studee_friendly_name,program_tuition_fees_per_year,program_delivery_method,state,...,university_number_of_international_students,university_number_of_nationalities,university_number_of_students,university_rank,university_type,_BATCH_ID__y,_BATCH_LAST_RUN__y,city_uuid,country_uuid,country_name
0,0000eb21-655d-4bd6-8641-e7f5898e35a3,ffd9efba-0b60-496e-b16f-f39fbb7c7832,6f082f72-51be-4944-baf4-41b4f1a81202,5db6e13f-8d81-11ec-b755-02cd10dd1184,Business Administration Diploma,6f082f72-51be-4944-baf4-41b4f1a81202,Business Administration,18130,On campus,published,...,367.0,40.0,9000.0,8936.0,public,2.0,2024-09-02 11:09:04,c25379f8-13ab-4848-9157-3b3bcb0c118b,d0ef05d1-5e72-4b53-a3fb-c87ae710f388,Canada
1,0027bdd9-8878-414d-b106-f00c03d6196b,c371d879-4eb3-431e-95bb-6489f5c36fc5,e20fe2b1-5517-4761-a401-6acad6aaa3c7,5fdc6e05-8d81-11ec-b755-02cd10dd1184,Project Management,e20fe2b1-5517-4761-a401-6acad6aaa3c7,NaN,9386,On campus,archived,...,3000.0,90.0,16600.0,1478.0,public,2.0,2024-09-02 11:09:04,4b02eafb-5ce8-4c2c-9a31-9f36058b10c8,d0ef05d1-5e72-4b53-a3fb-c87ae710f388,Canada
2,00462585-7b4d-49ad-81f1-3ddcd1a04ac5,ed0a6684-b8c1-4eb9-86f1-36493c9b4e1b,c0d94ff7-7378-4f49-b72e-e4fffc1ae189,ec3b723e-47a4-412e-b3eb-83909273a58b,Certificate in Youth Work Practice,c0d94ff7-7378-4f49-b72e-e4fffc1ae189,Youth Work Practice,3000,Blended learning,published,...,433.0,80.0,3095.0,805.0,private,2.0,2024-09-02 11:09:04,c97d3e0e-0a89-4562-b09e-6e32be978de1,dfa82742-fcd3-4733-a570-1dc9132a4356,USA
3,00b6a109-8634-414d-a405-3d3d920bf1ff,93dfada5-27a5-450d-9b0a-be2866634731,0b61620d-b2ff-473b-87bb-00b0a33f3997,5bd971e4-8d81-11ec-b755-02cd10dd1184,Business Administration: Business Analytics,0b61620d-b2ff-473b-87bb-00b0a33f3997,Business Administration: Business Analytics,8640,On campus,published,...,401.0,45.0,10000.0,2287.0,public,2.0,2024-09-02 11:09:04,4c11d10f-0b00-4bbc-bdfa-8bdb925b04e5,dfa82742-fcd3-4733-a570-1dc9132a4356,USA
4,00c1e5c2-8fa3-4da1-a803-a3e9adfb2d7e,17415f2c-c9f9-4517-a9fd-64a7d6f455fd,ebc763f9-2f33-45b4-9d49-e342d80ccf08,ee5490e6-c58f-4927-b027-fb2c9d913319,Public Relations – Corporate Communications,ebc763f9-2f33-45b4-9d49-e342d80ccf08,Public Relations: Corporate Communications,17789,On campus,published,...,11000.0,126.0,22000.0,4696.0,public,2.0,2024-09-02 11:09:04,79cca970-18f4-48e8-34a4-6f1b634e9a42,d0ef05d1-5e72-4b53-a3fb-c87ae710f388,Canada


In [15]:
# Rename columns to remove backticks and unnecessary characters for consistency
df_merged = df_merged_programs_app.rename(columns={
    'BIN_TO_UUID(`student_uuid`)': 'student_uuid',
    'BIN_TO_UUID(`program_uuid`)': 'program_uuid'
})

In [16]:
# Drop the duplicate 'program_uuid' column if both contain the same data
df_merged = df_merged.drop(columns=['program_uuid'])

In [17]:
print(df_merged.columns)


Index(['Student_App_UUID', 'Student_UUID', 'Program_UUID',
       'Program_Intake_UUID', 'program_name', 'program_studee_friendly_name',
       'program_tuition_fees_per_year', 'program_delivery_method', 'state',
       'keywords', 'university_uuid', 'program_qualification_period_uuid',
       'program_qualification_uuid', 'program_qualification_name',
       'BIN_TO_UUID(`city_uuid`)_x', 'BIN_TO_UUID(`click_uuid`)',
       'BIN_TO_UUID(`country_uuid`)', 'created_date_time',
       'isas_processed_date_time', 'BIN_TO_UUID(`program_level_uuid`)',
       'BIN_TO_UUID(`session_uuid`)', 'student_click_source',
       'BIN_TO_UUID(`subject_uuid`)', '_BATCH_ID__x', '_BATCH_LAST_RUN__x',
       'BIN_TO_UUID(`city_uuid`)_y', 'BIN_TO_UUID(`currency_uuid`)',
       'BIN_TO_UUID(`default_university_contact_uuid`)', 'is_commercial',
       'is_deposit_required', 'university_acceptance_rate',
       'university_allows_agents', 'university_decision_period',
       'university_location_address',
    

In [18]:
# Add click count per student and program combination
df_click_counts = df_merged.groupby(['Student_UUID', 'Program_UUID']).size().reset_index(name='click_count')

# Merge the click count data back into the main DataFrame
df_cleaned_with_clicks = pd.merge(df_merged, df_click_counts, on=['Student_UUID', 'Program_UUID'], how='left')

# Display the cleaned data
print(df_cleaned_with_clicks.head())


                       Student_App_UUID                          Student_UUID  \
0  0000eb21-655d-4bd6-8641-e7f5898e35a3  ffd9efba-0b60-496e-b16f-f39fbb7c7832   
1  0027bdd9-8878-414d-b106-f00c03d6196b  c371d879-4eb3-431e-95bb-6489f5c36fc5   
2  00462585-7b4d-49ad-81f1-3ddcd1a04ac5  ed0a6684-b8c1-4eb9-86f1-36493c9b4e1b   
3  00b6a109-8634-414d-a405-3d3d920bf1ff  93dfada5-27a5-450d-9b0a-be2866634731   
4  00c1e5c2-8fa3-4da1-a803-a3e9adfb2d7e  17415f2c-c9f9-4517-a9fd-64a7d6f455fd   

                           Program_UUID                   Program_Intake_UUID  \
0  6f082f72-51be-4944-baf4-41b4f1a81202  5db6e13f-8d81-11ec-b755-02cd10dd1184   
1  e20fe2b1-5517-4761-a401-6acad6aaa3c7  5fdc6e05-8d81-11ec-b755-02cd10dd1184   
2  c0d94ff7-7378-4f49-b72e-e4fffc1ae189  ec3b723e-47a4-412e-b3eb-83909273a58b   
3  0b61620d-b2ff-473b-87bb-00b0a33f3997  5bd971e4-8d81-11ec-b755-02cd10dd1184   
4  ebc763f9-2f33-45b4-9d49-e342d80ccf08  ee5490e6-c58f-4927-b027-fb2c9d913319   

                          

In [19]:
print(df_cleaned_with_clicks.columns)

Index(['Student_App_UUID', 'Student_UUID', 'Program_UUID',
       'Program_Intake_UUID', 'program_name', 'program_studee_friendly_name',
       'program_tuition_fees_per_year', 'program_delivery_method', 'state',
       'keywords', 'university_uuid', 'program_qualification_period_uuid',
       'program_qualification_uuid', 'program_qualification_name',
       'BIN_TO_UUID(`city_uuid`)_x', 'BIN_TO_UUID(`click_uuid`)',
       'BIN_TO_UUID(`country_uuid`)', 'created_date_time',
       'isas_processed_date_time', 'BIN_TO_UUID(`program_level_uuid`)',
       'BIN_TO_UUID(`session_uuid`)', 'student_click_source',
       'BIN_TO_UUID(`subject_uuid`)', '_BATCH_ID__x', '_BATCH_LAST_RUN__x',
       'BIN_TO_UUID(`city_uuid`)_y', 'BIN_TO_UUID(`currency_uuid`)',
       'BIN_TO_UUID(`default_university_contact_uuid`)', 'is_commercial',
       'is_deposit_required', 'university_acceptance_rate',
       'university_allows_agents', 'university_decision_period',
       'university_location_address',
    

In [20]:
# Select important columns, excluding the unnecessary ones and handling duplicates
df_cleaned = df_cleaned_with_clicks[[  
    'Student_UUID',  # Student identifier
    'university_uuid',  # Keep university UUID
    'Student_App_UUID',  # Student application UUID
    'student_click_source',  # Click source
    'Program_UUID',  # Program UUID
    'program_name',  # Program name
    'program_studee_friendly_name',  # Friendly program name
    'keywords',
    'program_tuition_fees_per_year',  # Tuition fees
    'university_rank',  # University rank
    'university_acceptance_rate',  # University acceptance rate
    'BIN_TO_UUID(`program_level_uuid`)',  # Program level UUID
    'click_count',  # Click count
    'program_qualification_period_uuid',  # Qualification period UUID
    'program_qualification_uuid',  # Program qualification UUID
    'program_qualification_name',  # Program qualification name
    'university_type',  # University type (public/private)
    'university_location_address',  # University location
    'country_name'  # Country name
]]

# Rename columns for easier reference
df_cleaned = df_cleaned.rename(columns={
    'Student_UUID': 'student_uuid',
    'Program_UUID': 'program_uuid',
    'university_uuid': 'university_uuid',
    'Student_App_UUID': 'student_application_uuid',
    'program_tuition_fees_per_year': 'tuition_fee',
    'BIN_TO_UUID(`program_level_uuid`)': 'program_level_uuid'
})

# Display the cleaned data
print(df_cleaned.head())

                           student_uuid                       university_uuid  \
0  ffd9efba-0b60-496e-b16f-f39fbb7c7832  882c90f2-7239-453c-f5c8-90a1a9c1f063   
1  c371d879-4eb3-431e-95bb-6489f5c36fc5  2da6553e-254a-418a-bb54-1495ea73b62c   
2  ed0a6684-b8c1-4eb9-86f1-36493c9b4e1b  b153aba4-ac7f-4a12-c854-ca3ed1682a6f   
3  93dfada5-27a5-450d-9b0a-be2866634731  3cd39092-e2d0-427f-9550-0ff9fb8d2684   
4  17415f2c-c9f9-4517-a9fd-64a7d6f455fd  66b23c3f-d421-494a-71d5-3fb9bcdd6d4e   

               student_application_uuid student_click_source  \
0  0000eb21-655d-4bd6-8641-e7f5898e35a3   University Overlay   
1  0027bdd9-8878-414d-b106-f00c03d6196b   University Overlay   
2  00462585-7b4d-49ad-81f1-3ddcd1a04ac5      Contact Capture   
3  00b6a109-8634-414d-a405-3d3d920bf1ff   University Overlay   
4  00c1e5c2-8fa3-4da1-a803-a3e9adfb2d7e      Contact Capture   

                           program_uuid  \
0  6f082f72-51be-4944-baf4-41b4f1a81202   
1  e20fe2b1-5517-4761-a401-6acad6aaa3c7   

In [21]:
# Convert 'tuition_fee' to numeric, coercing errors (non-numeric values will become NaN)
df_cleaned['tuition_fee'] = pd.to_numeric(df_cleaned['tuition_fee'], errors='coerce')

# Handle missing values in 'click_count', 'program_name', and 'tuition_fee'
df_cleaned['click_count'] = df_cleaned['click_count'].fillna(1)  # Replace NaN with 1 for missing click counts
df_cleaned['program_name'] = df_cleaned['program_name'].fillna('Unknown Program')  # Handle missing program names
df_cleaned['tuition_fee'] = df_cleaned['tuition_fee'].fillna(df_cleaned['tuition_fee'].mean())  # Fill missing tuition fees with the average

print(df_cleaned.head())

                           student_uuid                       university_uuid  \
0  ffd9efba-0b60-496e-b16f-f39fbb7c7832  882c90f2-7239-453c-f5c8-90a1a9c1f063   
1  c371d879-4eb3-431e-95bb-6489f5c36fc5  2da6553e-254a-418a-bb54-1495ea73b62c   
2  ed0a6684-b8c1-4eb9-86f1-36493c9b4e1b  b153aba4-ac7f-4a12-c854-ca3ed1682a6f   
3  93dfada5-27a5-450d-9b0a-be2866634731  3cd39092-e2d0-427f-9550-0ff9fb8d2684   
4  17415f2c-c9f9-4517-a9fd-64a7d6f455fd  66b23c3f-d421-494a-71d5-3fb9bcdd6d4e   

               student_application_uuid student_click_source  \
0  0000eb21-655d-4bd6-8641-e7f5898e35a3   University Overlay   
1  0027bdd9-8878-414d-b106-f00c03d6196b   University Overlay   
2  00462585-7b4d-49ad-81f1-3ddcd1a04ac5      Contact Capture   
3  00b6a109-8634-414d-a405-3d3d920bf1ff   University Overlay   
4  00c1e5c2-8fa3-4da1-a803-a3e9adfb2d7e      Contact Capture   

                           program_uuid  \
0  6f082f72-51be-4944-baf4-41b4f1a81202   
1  e20fe2b1-5517-4761-a401-6acad6aaa3c7   

In [22]:
# Cap tuition fees at the 95th percentile
cap_value = df_cleaned['tuition_fee'].quantile(0.95)
df_cleaned['tuition_fee'] = np.where(df_cleaned['tuition_fee'] > cap_value, cap_value, df_cleaned['tuition_fee'])

# Cap university rank at the 95th percentile
cap_value_rank = df_cleaned['university_rank'].quantile(0.95)
df_cleaned['university_rank'] = np.where(df_cleaned['university_rank'] > cap_value_rank, cap_value_rank, df_cleaned['university_rank'])

# Optionally, categorize the university ranks into tiers
df_cleaned['university_rank_category'] = pd.cut(
    df_cleaned['university_rank'], 
    bins=[0, 1000, 5000, 10000, 20000, np.inf],
    labels=['Top 1000', 'Top 5000', 'Top 10000', 'Top 20000', 'Others']
)

In [23]:
# Expanded subject_keywords based on the provided hierarchy, using Parent as the subject and Child, Grandchild, Great Grandchild as keywords
subject_keywords = {
    'Agriculture, Food & Animal Sciences': [
        'Agriculture', 'Agribusiness', 'Agricultural Leadership', 'Agricultural Development',
        'Agricultural Economics', 'Agricultural Production', 'Agricultural Sciences', 'Agroforestry',
        'Agronomy', 'Irrigation', 'Plant Science', 'Plant Pathology', 'Animal Science', 'Animal Behavior',
        'Human Animal Studies', 'Animal Breeding', 'Animal Care', 'Aquarium Science', 'Animal Genetics',
        'Animal Husbandry', 'Animal Nutrition', 'Animal Physiology', 'Animal Production', 'Dairy Production',
        'Companion Animal Science', 'Aquaculture', 'Culinary Arts', 'Chef', 'Baking and Pastry',
        'Commercial Cooking', 'Confectionery', 'Chocolatier', 'Equine Studies', 'Equine Management',
        'Equine Health', 'Food Industry', 'Food Services', 'Food Management', 'Food Production', 
        'Beer Brewing', 'Wine Studies', 'Plant-Based Foods', 'Food Science', 'Food Technology',
        'Dairy Foods Technology', 'Veterinary', 'Animal Health Technology', 'Veterinary Nursing', 
        'Aquatic Veterinary', 'Veterinary Pathobiology', 'Aquatic Pathobiology'
    ],
    'Applied Sciences & Mathematics': [
        'Astrobiology', 'Astronomy', 'Mathematics', 'Applied Mathematics', 'Actuarial Mathematics',
        'Financial Mathematics', 'Mathematical Analysis', 'Complex Analysis', 'Pure Mathematics',
        'Statistics', 'Biostatistics', 'Quantitative Science', 'Physics', 'Applied Physics', 'Astrophysics',
        'Cosmology', 'Chemical Physics', 'Theoretical Physics'
    ],
    'Architecture, Construction and Planning': [
        'Architecture', 'Architectural Studies', 'Architectural Technology', 'Construction Management',
        'Construction Technology', 'Drafting', 'Historic Preservation', 'Landscape Architecture',
        'Environmental Design', 'Sustainable Architecture', 'Sustainable Construction', 'Built Environments',
        'Building Automation', 'Building Services Engineering', 'Industrial Design', 'Interior Design',
        'Product Design', 'Ergonomic Studies', 'Ergonomic Design', 'Furniture Design', 'User Experience',
        'Surveying', 'Real Estate', 'Quantity Surveying', 'Urbanism', 'Energy Management', 'Environmental Planning',
        'Housing', 'Regional Planning', 'Transport Planning', 'Urban Administration', 'Urban Design', 
        'Urban Development', 'Urban Planning', 'Urban Studies'
    ],
    'Business & Management Studies': [
        'Accounting', 'Public Accounting', 'Strategic Accounting', 'Taxation', 'Auditing', 'Business',
        'Banking', 'Business Administration', 'Business Analytics', 'Business Management', 'Commercial Management',
        'Concrete Industry Management', 'Entertainment Management', 'Operations Management', 'Strategic Management',
        'Business Recovery', 'Customer Service', 'Entrepreneurship', 'Creative Enterprise', 'Event Management',
        'Hospitality Management', 'Human Resources', 'Innovation', 'International Business', 'Leadership',
        'Strategic Leadership', 'Logistics', 'Organizational Management', 'Product Development',
        'Recreation Management', 'Outdoor Recreation', 'Supply Chain Management', 'Tourism', 'Adventure Guide',
        'Tourism Management', 'Finance', 'Business Finance', 'Digital Currency', 'Financial Engineering',
        'Financial Information Systems', 'Financial Management', 'Financial Planning', 'Marketing', 'Advertising',
        'Creative Advertising', 'Digital Marketing', 'Market Research', 'Marketing Communications', 
        'Marketing Management', 'Merchandising', 'Project Management', 'Risk Management', 'Actuarial Science'
    ],
    'Communications & Media': [
        'Communication', 'Business Communication', 'Communication Management', 'Human Communication',
        'Mass Communication', 'Media Communication', 'Professional Communication', 'Professional Writing',
        'Public Relations', 'Strategic Communication', 'Technical Communication', 'Technical Writing',
        'Visual Communication', 'Journalism', 'Broadcast Journalism', 'Photojournalism', 'Sportscasting',
        'Media', 'Digital Media', 'Content Creation', 'Broadcasting', 'Captioning', 'Radio', 'Television',
        'Publishing', 'Narrative Studies'
    ],
    'Computer Science & IT': [
        'Computer Communications', 'Cloud Computing', 'Data Communication', 'Computer Networks',
        'Computer Information Systems', 'Business Information Systems', 'Computer Science', 'Applied Computer Science',
        'Bioinformatics', 'Computational Mathematics', 'Computer Applications', 'Computing', 'Informatics',
        'Artificial Intelligence', 'Machine Learning', 'Machine Vision', 'Speech Language Processing',
        'Computer Graphics', 'Computer Aided Drafting', 'Computational Modelling', 'Computer Generated Imagery',
        'Computer Graphics Technology', 'Computer Security', 'Cyber Security', 'Information Security',
        'Computer Technology', 'Computational Methods', 'Data Science', 'Analytics', 'Big Data', 'Data Mining',
        'Digital Forensics', 'Computer Forensics', 'Data Forensics', 'Forensic Data Analysis', 'Digital Humanities',
        'Human-Computer Interaction', 'Information Science', 'Information Management', 'Library Studies',
        'Research Methods', 'Software Development', 'Game Design', 'Character Design', 'Game Development',
        'Programming', 'Web Design', 'Web Development', 'Software Engineering', 'Virtual Reality', 'Telecommunications',
        'Wireless Systems'
    ],
    'Creative Arts & Design': [
        'Fashion', 'Fashion Design', 'Fashion Merchandising', 'Textiles', 'Jewellery Design', 'Graphic Design',
        'Graphic Communication', 'Graphic Novel', 'Illustration', 'Trades', 'Woodworking', 'Carpentry',
        'Cabinet Making', 'Visual Arts', 'Fine Arts', 'Drawing', 'Painting', 'Photography', 'Plastic Arts',
        'Ceramics', 'Sculpture', 'Modelmaking', 'Applied Arts', 'Studio Art', 'Printmaking', 'Multimedia'
    ],
    'Performing Arts': [
        'Acting', 'Screen Acting', 'Circus Arts', 'Contemporary Circus', 'Comedy', 'Dance', 'Ballet',
        'Choreography', 'Film', 'Animation', '2D Animation', '3D Animation', 'Filmmaking', 'Film Production',
        'Documentary', 'Screenwriting', 'Film Studies', 'Visual Effects', '3D Visualization', 'Music', 'Composition',
        'Conducting', 'Choral Conducting', 'Jazz Studies', 'Jazz Performance', 'Music Business', 'Music Performance',
        'Classical String Performance', 'Concert Performance', 'Guitar Performance', 'Ensemble Performance',
        'Organ Performance', 'Piano Performance', 'Music Production', 'Music Technology', 'Musical Instruments',
        'Bassoon', 'Cello', 'Clarinet', 'Double Bass', 'Euphonium', 'Flute', 'Guitar', 'Harp', 'Oboe', 'Percussion',
        'Saxophone', 'Trombone', 'Viola', 'Violin', 'Music Theory', 'Songwriting', 'Vocal Performance', 'Production',
        'Costume Design', 'Lighting Design', 'Prop Making', 'Scenography', 'Sound Design', 'Theatre Sound',
        'Sound Engineering', 'Theatre', 'Applied Theatre', 'Physical Theatre', 'Theatre Studies', 'Dramaturgy',
        'Directing', 'Stage Management', 'Playwriting'
    ],

    'Education & Training': [
        'Coaching', 'Education', 'Adult Education', 'Agricultural Education', 'Art Education',
        'Biology Education', 'Business Education', 'Curriculum and Instruction', 'Chemistry Education',
        'Dance Education', 'Early Childhood', 'Education Research', 'Educational Leadership',
        'Educational Technology', 'Elementary Education', 'Extension Education', 'History Education',
        'Language Arts', 'Language Education', 'French Teaching', 'Spanish Teaching', 'TESOL',
        'Learning Development', 'Music Education', 'Choral Education', 'Instrument Education',
        'Mathematics Education', 'Nursing Education', 'Outdoor Education', 'Physical Education',
        'Physics Education', 'Reading Education', 'Religious Education', 'Science Education',
        'Secondary Education', 'Social Sciences Education', 'Social Studies Education', 'Theatre Education',
        'Special Education', 'Autism', 'Deaf Education', 'Education Intervention', 'Deafblind Intervener',
        'Exceptional Child Education', 'Teaching', 'Instructional Design', 'Literacy', 'Pedagogy',
        'Teaching Assistant'
    ],
    'Engineering & Aviation': [
        'Acoustical Engineering', 'Noise Control', 'Aerospace Engineering', 'Aerospace Manufacturing',
        'Aerospace Materials', 'Aerospace Propulsion', 'Aeronautical Engineering', 'Astronautical Engineering',
        'Avionic Engineering', 'Drone Engineering', 'Spatial Science', 'Agricultural Engineering',
        'Automotive Engineering', 'Automotive Maintenance', 'Automotive Repair', 'Collision and Refinishing',
        'Engine Repair', 'Hybrid Automotive Systems', 'Diesel Mechanics', 'Aviation', 'Air Traffic Control',
        'Airport Management', 'Aviation Management', 'Aviation Technology', 'Flight Attendant',
        'Professional Pilot', 'Bioengineering', 'Biomedical Engineering', 'Chemical Engineering',
        'Nanotechnology', 'Nuclear Engineering', 'Petroleum Engineering', 'Civil Engineering',
        'Civil Infrastructure', 'Construction Engineering', 'Environmental Engineering', 'Geotechnics',
        'Structural Engineering', 'Architectural Engineering', 'Transport Engineering', 'Truck Driving',
        'Computer Engineering', 'Cybernetics', 'Design Engineering', 'Computer Aided Design',
        'Electrical Engineering', 'Electronic Engineering', 'Microelectronics', 'Energy Engineering',
        'Renewable Energy Engineering', 'Engineering Technology', 'Industrial Engineering',
        'Manufacturing Engineering', 'Automated Manufacturing', 'Computer Numerical Control', 'Welding',
        'Mold Making', 'Facility Maintenance', 'Millwright', 'Plumbing', 'Quality Management',
        'Materials Engineering', 'Composite Technology', 'Composite Technology Drafting',
        'Composite Technology Electrical', 'Composite Technology Metals', 'Composite Structures',
        'Composite Technology Wood', 'Metallurgy', 'Mechanical Engineering', 'Air Conditioning Technology',
        'Continuum Mechanics', 'Mechatronics', 'Robotics', 'Medical Engineering', 'Mining Engineering',
        'Nanoengineering', 'Photonics', 'Systems Engineering', 'Technology Engineering', 'Technology Drafting'
    ],
    'Environmental & Earth Sciences': [
        'Biodiversity', 'Forest Management', 'Forestry', 'Sustainability', 'Sustainable Development',
        'Conservation', 'Biodiversity Conservation', 'Conservation Management', 'Conservation Land Management',
        'Forest Conservation', 'Marine Conservation', 'Wildlife Conservation', 'Earth Sciences',
        'Geophysical Sciences', 'Geochemistry', 'Mineralogy', 'Gemology', 'Paleontology', 'Paleobiology',
        'Soil Science', 'Agricultural Soil Science', 'Ecology', 'Agroecology', 'Desert Ecology',
        'Urban Ecology', 'Urban Agroecology', 'Restoration Ecology', 'Environmental Studies',
        'Ecosystem Science', 'Environmental Management', 'Air Quality Management', 'Coastal Environmental Science',
        'Waste Management', 'Environmental Science', 'Environmental Technology', 'Geography', 'Biogeography',
        'Geology', 'Geomatics', 'Geoinformatics', 'Meteorology', 'Cartography', 'Oceanography',
        'Marine System Science', 'Hydrology', 'Water Management', 'Water Resources Engineering',
        'Water Treatment', 'Marine Science', 'Marine Biology'
    ],
    'Humanities & Law': [
        'Anthropology', 'Cultural Anthropology', 'Ethnology', 'Ethnomusicology', 'Evolutionary Anthropology',
        'Forensic Anthropology', 'Social Anthropology', 'Archaeology', 'Museum Studies', 'Curating',
        'Geoarchaeology', 'Cultural Studies', 'African-American Studies', 'African Studies', 'American Studies',
        'Southern Studies', 'Asia Pacific Studies', 'New Zealand Studies', 'Pacific Policing', 'Samoan Studies',
        'Asian Studies', 'Canadian Studies', 'Caribbean Studies', 'Chinese Studies', 'Classical Studies',
        'Colonial Studies', 'Cultural Resource Management', 'Egyptology', 'European Studies', 'Germanic Studies',
        'Scandinavian Studies', 'Slavic Studies', 'First Nation Studies', 'Native-American Studies',
        'Maori Studies', 'Indigenous Development', 'Indigenous Psychologies', 'First Nations Education',
        'First Nations Language Instruction', 'Popular Culture', 'Greek', 'Irish Studies', 'Italian Studies',
        'Jewish Studies', 'Latin American Studies', 'Mexican Studies', 'Martial Arts Studies',
        'Middle Eastern Studies', 'Arab Studies', 'Celtic Studies', 'General Studies', 'General Arts',
        'Liberal Arts', 'General Sciences', 'Metrology', 'Foundation Studies', 'ESL', 'Pathway Programs',
        'History', 'American History', 'Ancient History', 'Art History', 'Classical Civilization', 'Modern History',
        'Renaissance Studies', 'Holocaust and Genocide Studies', 'Heritage Studies', 'Justice Services',
        'Criminal Justice', 'Criminal Justice Administration', 'Homeland Security', 'Juvenile Justice',
        'Police Studies', 'Languages', 'Arabic', 'Asian Language', 'Chinese', 'French', 'German', 'Italian',
        'Irish', 'Japanese', 'Korean', 'Latin', 'Maori Language', 'Pacific Languages', 'Portuguese',
        'Russian', 'Sign Language', 'Spanish', 'English Language', 'Literature', 'Critical Theory',
        'Comparative Literature', 'Contemporary Literature', 'Creative Writing', 'Gothic Literature',
        'English Literature', 'Poetry', 'Linguistics', 'Interpreting', 'Conference Interpreting', 'Legal Interpreting',
        'Phonetics', 'Translation', 'Law', 'Business Law', 'Criminal Law', 'Employment Law', 'Environmental Law',
        'Health Law', 'Immigration Law', 'Intellectual Property Law', 'International Law', 'Labor Law',
        'Legal Practice', 'Legal Studies', 'Maritime Law', 'Philosophy', 'Aesthetics', 'Bioethics', 'Ethics',
        'Population Studies', 'Professional Studies', 'Theology', 'Religious Studies', 'Biblical Studies',
        'Buddhist Studies', 'Catholic Studies', 'Christian Studies', 'Islamic Studies', 'Pastoral Studies'
    ],
    'Life & Natural Sciences': [
        'Biological Sciences', 'Anatomy', 'Cellular Biology', 'Biotechnology', 'Medical Biotechnology',
        'Bioprocessing', 'Biology', 'Forensic Biology', 'Human Biology', 'Animal Biology', 'Entomology',
        'Zoology', 'Primatology', 'Genetics', 'Molecular Biology', 'Genomics', 'Neuroscience', 'Macrobiology',
        'Microbiology', 'Bacteriology', 'Medical Microbiology', 'Chemistry', 'Biochemistry', 'Medical Biochemistry',
        'Cosmetic Science', 'Color Science', 'Polymer Chemistry', 'Pharmaceutical Sciences', 'Pharmacology'
    ],
    'Medicine & Health Sciences': [
        'Allied Health', 'Audiology', 'Histotechnology', 'Laboratory Science', 'Medical Laboratory Science', 
        'Chemical Laboratory Science', 'Medical Technology', 'Medical Transcription', 'Occupational Health', 
        'Occupational Safety', 'Optometry', 'Orthopaedics', 'Physiotherapy', 'Chiropractic', 'Physical Therapy Assistant', 
        'Radiography', 'Speech Pathology', 'Veterinary Technology', 'Biomedical Sciences', 'Counseling', 
        'Addiction Counseling', 'Career Counseling', 'Clinical Counseling', 'Educational Counseling', 'Family Therapy', 
        'Genetic Counseling', 'Professional Counseling', 'Rehabilitation Counseling', 'Therapeutic Counseling', 
        'Cosmetology', 'Esthetician', 'Nail Care', 'Hair Stylist', 'Dentistry', 'Dental Assisting', 'Dental Hygiene', 
        'Dental Surgery', 'Orthodontics', 'Periodontics', 'Disorders', 'Speech-Language Pathology', 'Communication Disorders', 
        'Diagnosis', 'Endoscopy', 'Diagnostic Imaging', 'Echocardiography', 'Ultrasonography', 'Tomography', 
        'Roentgen-Endovascular Diagnostics', 'Neurodiagnostic', 'Healthcare', 'Clinical Integration', 'Clinical Research', 
        'Digital Health', 'Hygiene', 'Health Communication', 'Health Informatics', 'Consumer Health Informatics', 
        'Medical Coding', 'Healthcare Administration', 'Healthcare Management', 'Hospital Administration', 
        'Hospital Management', 'Medical Assistance', 'Medical Humanities', 'Midwifery', 'Mortuary Science', 
        'Occupational Therapy', 'Podiatry', 'Public Health', 'Environmental Health', 'Rehabilitation Studies', 'Toxicology', 
        'Medicine', 'Anesthesiology', 'Anesthesia', 'Cardiology', 'Emergency Medicine', 'Dermatology', 'Dermatovenerology', 
        'Gynecology', 'Obstetrics', 'Hematology', 'Transfusiology', 'Immunology', 'Allergology', 'Immunotherapy', 
        'Endocrinology', 'Epidemiology', 'Infectious disease', 'Gastroenterology', 'Medical Management', 'Neurology', 
        'Neurosurgery', 'Nephrology', 'Pediatrics', 'Neonatology', 'Pediatric Endocrinology', 'Pediatric Oncology', 
        'Pulmonology', 'Phthisiology', 'Rheumatology', 'Surgery', 'Clinical Perfusion', 'Oncology', 'Radiation Oncology', 
        'Cancer Immunotherapy', 'Oncology Research', 'Ophthalmology', 'Otorhinolaryngology', 'Osteopathy', 
        'Palliative Care', 'Urology', 'Traumatology', 'Nursing', 'Adult Nursing', 'Child Nursing', 'Family Nursing', 
        'Mental Health Nursing', 'Midwifery', 'Nurse Anesthesia', 'Nurse Practitioner', 'Nursing Administration', 
        'Nursing Informatics', 'Nutrition', 'Physiology', 'Exercise Physiology', 'Human Physiology', 'Kinesiology', 
        'Medical Physiology', 'Orthotics', 'Prosthetics', 'Psychiatry', 'Psychology', 'Applied Psychology', 
        'Sports Psychology', 'Behavioral Psychology', 'Behavioral Neuroscience', 'Cyberpsychology', 'Child Psychology', 
        'Clinical Psychology', 'Cognitive Science', 'Cognitive Psychology', 'Counseling Psychology', 'Criminal Psychology', 
        'Educational Psychology', 'Environmental Psychology', 'Forensic Psychology', 'Thanatology', 'Industrial Psychology', 
        'Positive Psychology', 'Psychoanalysis', 'Psychobiology', 'Psychophysiology', 'Neuropsychology', 
        'Organizational Psychology', 'Social Psychology', 'Psychotherapy', 'Therapeutics', 'Acupuncture', 
        'Holistic Medicine', 'Massage Therapy', 'Recreation Therapy', 'Respiratory Therapy', 'Sports', 'Biomechanics', 
        'Physical Fitness', 'Sports Administration', 'Sports Management', 'Sports Medicine', 'Sports Rehabilitation', 
        'Sports Training', 'Football Training'
    ],
        'Social Studies': [
        'Criminology', 'Forensic Science', 'Forensic Accounting', 'Forensic Analysis', 
        'Forensic Identification', 'Forensic Nursing', 'Community Studies', 'Community Development', 
        'Community Research', 'Community Safety', 'Consumer Studies', 'Disability Studies', 'Deaf Studies', 
        'Economics', 'Applied Economics', 'Business Economics', 'Econometrics', 'Environmental Economics', 
        'Development Economics', 'Financial Economics', 'International Economics', 'Human Development', 
        'Gender Studies', 'Women\'s Studies', 'Human Rights', 'Diversity Studies', 'International Development', 
        'International Security', 'International Relations', 'Justice Studies', 'Political Science', 
        'Diplomacy', 'Dispute Resolution', 'Emergency Management', 'Governance', 'Nationalism Studies', 
        'Peace Studies', 'Public Administration', 'Public Management', 'Regulatory Affairs', 'Public Safety', 
        'Disaster Studies', 'Nonprofit Studies', 'Politics', 'Public Policy', 'Health Policy', 'Education Policy', 
        'Environmental Policy', 'Coastal Ocean Policy', 'Social Policy', 'Social Science', 'Social Research', 
        'Social Work', 'Addiction Studies', 'Addiction Rehabilitation', 'Child Studies', 'Charity Development', 
        'Clinical Social Work', 'Collaborative Health', 'Gerontology', 'Dementia Care', 'Family Studies', 
        'Mental Health', 'Social Impact', 'Youth Studies', 'Sociology', 'Critical Ethnic Studies', 'Migration Studies', 
        'Educational Sociology', 'Sexology', 'Human Sexuality', 'Sociology of Globalization', 'Victimology'
    ]
  
}


In [24]:
# Save the DataFrame as a CSV file
df_cleaned.to_csv('assigned_subjects.csv', index=False)

In [25]:
import re
import ast  # For parsing the string representation of a list

# Function to assign subjects based on program name or keyword list
def assign_subject(program_name, keywords):
    # Check program name against expanded subject_keywords
    for subject, subject_list in subject_keywords.items():
        # Check program name first
        if any(keyword.lower() in program_name.lower() for keyword in subject_list):
            return subject

        # Check the keywords column if available
        if keywords:
            # Ensure keywords are a list of dictionaries (if it's a string, parse it)
            if isinstance(keywords, str):
                try:
                    keywords = ast.literal_eval(keywords)  # Convert the string representation to a list
                except (ValueError, SyntaxError):
                    return 'Other'  # If parsing fails, return 'Other'

            # Assuming keywords are stored as a list of dictionaries
            for keyword_dict in keywords:
                keyword = keyword_dict.get('keyword', '')
                if any(re.search(rf'\b{kw}\b', keyword, re.IGNORECASE) for kw in subject_list):
                    return subject

    return 'Other'

# Example of how to use the function
df_cleaned['assigned_subject'] = df_cleaned.apply(
    lambda row: assign_subject(row['program_name'], row['keywords']),
    axis=1
)

In [26]:
print(df_cleaned[['program_name', 'keywords', 'assigned_subject']].head(10))

                                  program_name  \
0              Business Administration Diploma   
1                           Project Management   
2           Certificate in Youth Work Practice   
3  Business Administration: Business Analytics   
4  Public Relations – Corporate Communications   
5                        Personalised Medicine   
6                        Personalised Medicine   
7                        Personalised Medicine   
8                        Personalised Medicine   
9                        Personalised Medicine   

                                            keywords  \
0  [{"keyword": "Business Administration", "subje...   
1                                                 []   
2  [{"keyword": "Youth Work Practice", "subject":...   
3  [{"keyword": "Business Administration", "subje...   
4  [{"keyword": "Public Relations", "subject": {"...   
5                                                 []   
6                                                 []   
7

In [27]:
# Fill missing values for tuition fee and university rank without using inplace
df_cleaned['tuition_fee'] = df_cleaned['tuition_fee'].fillna(df_cleaned['tuition_fee'].mean())
df_cleaned['university_rank'] = df_cleaned['university_rank'].fillna(df_cleaned['university_rank'].mean())

# Normalize tuition fee and university rank
scaler = MinMaxScaler()

# Normalize the columns
df_cleaned['normalized_tuition_fee'] = scaler.fit_transform(df_cleaned[['tuition_fee']])
df_cleaned['normalized_university_rank'] = scaler.fit_transform(df_cleaned[['university_rank']])


In [28]:
# Handle NaN values by replacing with 0 or column mean before applying cosine_similarity
df_cleaned['normalized_university_rank'] = df_cleaned['normalized_university_rank'].fillna(0)
df_cleaned['normalized_tuition_fee'] = df_cleaned['normalized_tuition_fee'].fillna(0)

In [29]:
print(df_cleaned[['tuition_fee', 'normalized_tuition_fee', 'university_rank', 'normalized_university_rank']].head())

   tuition_fee  normalized_tuition_fee  university_rank  \
0      18130.0                0.556252           8936.0   
1       9386.0                0.272975           1478.0   
2       3000.0                0.066089            805.0   
3       8640.0                0.248807           2287.0   
4      17789.0                0.545205           4696.0   

   normalized_university_rank  
0                    0.599651  
1                    0.099181  
2                    0.054020  
3                    0.153469  
4                    0.315125  


In [30]:
# Step 3: Calculate Similarity for Content-based Filtering - Encode subjects and qualifications
subject_encoder = OneHotEncoder(sparse_output=False)  # Use dense array output
subject_encoded = subject_encoder.fit_transform(df_cleaned[['assigned_subject']])

print(f"Shape of subject_encoded: {subject_encoded.shape}")

Shape of subject_encoded: (2339, 16)


In [31]:
# Adjust the weights for different features (subjects, tuition fees, rank, etc.)
weight_subject = 0.4
weight_tuition = 0.3
weight_rank = 0.3

In [32]:
# Combine the weighted content features
combined_features = np.hstack((
    weight_subject * subject_encoded,
    weight_tuition * df_cleaned[['normalized_tuition_fee']].values,
    weight_rank * df_cleaned[['normalized_university_rank']].values
))

In [33]:
# Step 1: Calculate the content similarity matrix using cosine similarity
content_similarity_matrix = cosine_similarity(combined_features)

# Step 2: Store the similarity matrix in a DataFrame for easier access
# Rows and columns represent 'program_uuid'
content_similarity_df = pd.DataFrame(content_similarity_matrix, 
                                     index=df_cleaned['program_uuid'], 
                                     columns=df_cleaned['program_uuid'])

print(f"Shape of combined features: {combined_features.shape}")

Shape of combined features: (2339, 18)


In [78]:
similarity_score = content_similarity_df.loc[applied_program_uuid, recommended_program_uuid]
adjusted_rating = estimated_rating * similarity_score

In [79]:
# 2. Normalize click_count to a range of 1 to 5
scaler = MinMaxScaler(feature_range=(1, 5))
df_cleaned['normalized_click_count'] = scaler.fit_transform(df_cleaned[['click_count']])

# Print to confirm normalization
print("First 5 rows after normalizing click counts:")
print(df_cleaned[['click_count', 'normalized_click_count']].head())

First 5 rows after normalizing click counts:
   click_count  normalized_click_count
0            1                     1.0
1            1                     1.0
2            1                     1.0
3            1                     1.0
4            1                     1.0


In [80]:
# Set the rating scale based on the click count (assuming it's the implicit feedback)
reader = Reader(rating_scale=(1, 5))  # Since we normalized to 1-5, set this range
data = Dataset.load_from_df(df_cleaned[['student_uuid', 'program_uuid', 'normalized_click_count']], reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2)

# 4. Define and train the SVD model (with tuned parameters)
svd_model = SVD(n_factors=100, n_epochs=100, lr_all=0.003, reg_all=0.1)
svd_model.fit(trainset)

# Evaluate the model on the test set
predictions = svd_model.test(testset)

# 5. Print the first 5 predictions for validation
print("SVD Predictions (First 5 rows):")
for i, prediction in enumerate(predictions[:5]):
    print(f"Prediction {i+1}: User {prediction.uid}, Program {prediction.iid}, Estimated Rating {prediction.est}, True Rating {prediction.r_ui}")

SVD Predictions (First 5 rows):
Prediction 1: User 00288a5f-3bb0-47bf-8844-a54b848558bc, Program 3d3b436d-913e-4dee-810b-296968d3dcfe, Estimated Rating 1.3245082755162934, True Rating 1.2857142857142856
Prediction 2: User 712ca873-3033-4562-b439-e9a14cc12dd5, Program 22034e77-f74b-4735-a69d-c57089805ef2, Estimated Rating 1.214664298086808, True Rating 1.0952380952380953
Prediction 3: User bac40aa2-0d5b-4b4e-b8bd-0216d970f3f0, Program 1195ee18-f359-430f-8a71-a525beb50ccb, Estimated Rating 1.842075793438701, True Rating 1.0
Prediction 4: User ba132f22-5617-49ed-9a94-35458d5ccd5a, Program 7b0766a2-6327-42cd-a64a-73415680c096, Estimated Rating 1.6935786701497635, True Rating 1.0
Prediction 5: User 83299264-62c0-47c7-be30-08f5cee16e02, Program 40293988-904f-49f9-a7ef-8e875a64436b, Estimated Rating 4.903126217616203, True Rating 5.0


In [81]:
from surprise import accuracy

# Compute RMSE and MAE on the test set
rmse = accuracy.rmse(predictions, verbose=True)
mae = accuracy.mae(predictions, verbose=True)

RMSE: 0.3658
MAE:  0.2446


In [82]:
# Create a dictionary mapping program_uuid to program_name from df_cleaned
program_uuid_to_name = df_cleaned.set_index('program_uuid')['program_name'].to_dict()

# Function to map program_uuid to program_name
def get_program_name(program_uuid):
    return program_uuid_to_name.get(program_uuid, 'Unknown Program')

# Print predictions with program names
print("SVD Predictions (First 5 rows):")
filtered_predictions = []  # To store filtered predictions
for i, prediction in enumerate(predictions):
    student_uuid = prediction.uid
    recommended_program_uuid = prediction.iid
    estimated_rating = prediction.est
    true_rating = prediction.r_ui
    
    # Get the applied program UUID from the test set
    applied_program_uuid = next((t[1] for t in testset if t[0] == student_uuid), None)
    
    # If the recommended program is the same as the applied program, skip it
    if recommended_program_uuid == applied_program_uuid:
        continue  # Skip this recommendation as it's the same program

    # Get the program names
    applied_program_name = get_program_name(applied_program_uuid)
    recommended_program_name = get_program_name(recommended_program_uuid)
    
    # Print the filtered prediction details
    print(f"Prediction {i+1}:")
    print(f"User: {student_uuid}")
    print(f"Applied Program: {applied_program_name} (UUID: {applied_program_uuid})")
    print(f"Recommended Program: {recommended_program_name} (UUID: {recommended_program_uuid})")
    print(f"Estimated Rating: {estimated_rating}, True Rating: {true_rating}")
    print("-" * 80)
    
    # Add to filtered list
    filtered_predictions.append({
        'student_uuid': student_uuid,
        'applied_program_uuid': applied_program_uuid,
        'recommended_program_uuid': recommended_program_uuid,
        'estimated_rating': estimated_rating,
        'true_rating': true_rating,
        'applied_program_name': applied_program_name,
        'recommended_program_name': recommended_program_name
    })

# Print how many valid recommendations were found
print(f"Number of valid recommendations (not matching applied programs): {len(filtered_predictions)}")

SVD Predictions (First 5 rows):
Prediction 14:
User: 83299264-62c0-47c7-be30-08f5cee16e02
Applied Program: Applied Computer Science (UUID: 40293988-904f-49f9-a7ef-8e875a64436b)
Recommended Program: Automotive Hybrid and Electrical Vehicle (UUID: 453ff698-5044-4a6d-8d92-4e51b026d1fc)
Estimated Rating: 4.902101513589569, True Rating: 5.0
--------------------------------------------------------------------------------
Prediction 16:
User: 10a7dc7a-dedf-4cc1-9652-d298291028b1
Applied Program: Social Sciences (UUID: aec1d172-2ab9-4dc4-9c87-5036d79afa12)
Recommended Program: Accounting, Controlling and Financial Management (UUID: d7c440ed-90f2-4878-a933-bfd21872689a)
Estimated Rating: 2.826977246692647, True Rating: 2.714285714285714
--------------------------------------------------------------------------------
Prediction 17:
User: 10a7dc7a-dedf-4cc1-9652-d298291028b1
Applied Program: Social Sciences (UUID: aec1d172-2ab9-4dc4-9c87-5036d79afa12)
Recommended Program: Bachelor's Degree Progra

In [42]:
# Convert the filtered_predictions list to a DataFrame
predictions_df = pd.DataFrame(filtered_predictions)

# Define the output CSV file path (Kaggle will store it under '/kaggle/working/' directory)
csv_file_path = '/kaggle/working/recommendation_results.csv'

# Save the DataFrame to a CSV file
predictions_df.to_csv(csv_file_path, index=False)

# Display a message to confirm the file was saved
print(f"Filtered predictions have been saved to {csv_file_path}")

Filtered predictions have been saved to /kaggle/working/recommendation_results.csv


In [38]:
for prediction in predictions[:10]:
    print(f"Student {prediction.uid} - Program {prediction.iid}: Actual Rating {prediction.r_ui}, Predicted Rating {prediction.est}")

Student 855dd114-acc3-4765-90bf-21e8ce922656 - Program 1fcd0f5c-69f4-486f-8cd0-a275cb79752c: Actual Rating 1.0952380952380953, Predicted Rating 1.1918065789309913
Student d3c537cd-1898-4601-bde6-e2e984ea7ee5 - Program facd1e13-71d9-4669-aed0-5323e6a6373b: Actual Rating 1.0, Predicted Rating 1.8319462472321906
Student 16aef805-2130-4dc4-a89d-d7836a750f52 - Program b4438003-e0ce-4c14-a211-629d65427e56: Actual Rating 1.0952380952380953, Predicted Rating 1.2353939490748302
Student 10a7dc7a-dedf-4cc1-9652-d298291028b1 - Program adf2fdd5-9366-4a5c-8e5c-3e8d08a5f12a: Actual Rating 2.714285714285714, Predicted Rating 2.501487505001291
Student 855dd114-acc3-4765-90bf-21e8ce922656 - Program 32c45144-8c51-4907-973d-d987e7f7b24c: Actual Rating 1.0952380952380953, Predicted Rating 1.1919713189970333
Student 4a962284-d33a-4ac1-b851-a7e7abed8c4d - Program 63963284-cf68-4edf-b248-5eae88653af1: Actual Rating 1.7619047619047619, Predicted Rating 1.702780471945503
Student 8f449ab8-5e50-4472-ab83-42c267dc